# Model Training with Multithreading

## Imports, Trainingdata- and Validationdata-Setup

In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.utils.vis_utils import plot_model
import sklearn
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import math
import ipywidgets as widgets
#!/usr/bin/python
import queue
import threading
import time


%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reproducible.
#from tensorflow import set_random_seed
from numpy.random import seed
seed(1)

train = pd.read_csv('TestData/train.csv',',')

# train.describe()

split_fraction = 0.70
train_split = int(split_fraction * int(train.shape[0]))
step = 1

#past = 0
#future = 8
learning_rate = 0.0005
batch = 256
epochs = 100
mean = 0
std = 0

def normalize(data, train_split):
    global mean
    global std
    data_mean = data[:train_split].mean(axis=0)
    mean = data_mean
    data_std = data[:train_split].std(axis=0)
    std = data_std
    return (data - data_mean) / data_std

titles = []
for c in train.columns:
    titles.append(c);
    
features = train[titles]
features = normalize(features.values, train_split)
features = pd.DataFrame(features)

train_data = features.loc[0 : train_split - 1] #Training Data
val_data = features.loc[train_split:] #Validation Data

#start = past + future
start = 0
end = start + train_split

x_train = train_data[[i for i in range(9)]].values
y_train = features.iloc[start:end][[9]]

#sequence_length = int(past / step)
sequence_length = 1

dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch
)

label_start = train_split
valRange = int(train.shape[0]) - train_split

# x_val = val_data.iloc[[i for i in range(valRange)]].values
x_val = val_data[[i for i in range(9)]].values
# x_val = val_data.iloc[[i for i in range(49)]].values
y_val = features.iloc[label_start:][[9]]

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch
)

for batch in dataset_train.take(1):
    inputs, targets = batch
    
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (256, 1, 9)
Target shape: (256, 1)


## Define important Functions

In [2]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

def denormalize(value):
    data_mean = mean[9]
    data_std = std[9]
    return value*data_std+data_mean

def scheduler(epoch, lr):
    return 0.001
    if lr > 0.004:
        return lr - 0.0002
    else:
        if lr > 0.0004:
            return lr - 0.000001
        else:            
            return 0.0001
#     if epoch < 10:
#         return lr
#     else:
#         if lr < 0.002:
#             return lr - 0.00001
#         else:
#             return lr - 0.0015
#     if lr < 0.002:
#         return lr - 0.00001
#     else:
#         return lr

es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10)

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)


# array that holds every model --> models.append(...) is needed after every model definition
models = []
# array that holds every training output --> outs.append(...) is needed after every output definition
outs = []
# array that holds every modelcheckpoint callback --> modelcheckpoints.append(...) is needed after every modelcheckpoint callback definition
modelcheckpoints = []

## Define Variables for Models and Training

In [3]:
learning_rate = 0.01
epochs = 4000

#define input layer for every model
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))

## Model 1 - Dense with activation 'tanh'

In [4]:
modelckpt_callback_1 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath="kerasModel1/modelCheckpoint_1.h5",
    verbose=1,
    save_weights_only=True,
    save_best_only=True
)

dropout1 = keras.layers.Dropout(0.2)(inputs)
dense1 = keras.layers.Dense(10, activation='tanh', kernel_constraint=keras.constraints.MaxNorm(max_value=3, axis=0))(dropout1)
dropout2 = keras.layers.Dropout(0.2)(dense1)
outputs = keras.layers.Dense(1)(dropout2)
model1 = keras.Model(inputs=inputs, outputs=outputs)
model1.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8), loss=keras.losses.MeanAbsoluteError(), metrics=keras.metrics.MeanSquaredError())
model1.summary()
model1.save("kerasModel1")
models.append(model1)
modelcheckpoints.append(modelckpt_callback_1)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 9)]            0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 9)              0         
_________________________________________________________________
dense (Dense)                (None, 1, 10)             100       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 10)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instr

### Output Training Model 1

In [5]:
out1 = widgets.Output(layout={'border': '1px solid black'})
outs.append(out1)
out1

Output(layout=Layout(border='1px solid black'))

## Model 2 - LSTM

In [6]:
modelckpt_callback_2 = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath="kerasModel2/modelCheckpoint_2.h5",
    verbose=1,
    save_weights_only=True,
    save_best_only=True
)

dropout1 = keras.layers.Dropout(0.2)(inputs)
dense1 = keras.layers.LSTM(10, return_sequences=True, kernel_constraint=keras.constraints.MaxNorm(max_value=3, axis=0))(dropout1)
dropout2 = keras.layers.Dropout(0.2)(dense1)
outputs = keras.layers.Dense(1)(dropout2)
learning_rate = 0.01
model2 = keras.Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer=keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8), loss=keras.losses.MeanAbsoluteError(), metrics=keras.metrics.MeanSquaredError())
model2.summary()
model2.save("kerasModel2")
models.append(model2)
modelcheckpoints.append(modelckpt_callback_2)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 9)]            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 9)              0         
_________________________________________________________________
lstm (LSTM)                  (None, 1, 10)             800       
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 10)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 1)              11        
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: kerasModel2\assets


### Output Training Model 2

In [7]:
out2 = widgets.Output(layout={'border': '1px solid black'})
outs.append(out2)
out2

Output(layout=Layout(border='1px solid black'))

## Multithreading/Start of Training

In [8]:
exitFlag = 0

switcherModels = {}
i = 1
for model in models:
    switcherModels[i] = model
    i += 1
    
switcherOuts = {}
j = 1
for output in outs:
    switcherOuts[j] = output
    j += 1
    
switcherCheckpoints = {}
k = 1
for checkpoint in modelcheckpoints:
    switcherCheckpoints[k] = checkpoint
    k += 1
    
numberOfModels = 2

for i in range(numberOfModels):
    index = i + 1
    with switcherOuts.get(index):
        currentModel = switcherModels.get(index)
        history = currentModel.fit(
            dataset_train,
            epochs=epochs,
            validation_data=dataset_val,
            callbacks=[es_callback,switcherCheckpoints.get(index), lr_scheduler]
        )
        
        visualize_loss(history, "Training and Validation loss")
        
        for x, y in dataset_val.take(1):
            show_plot(
                [x[0][:, 0].numpy(), y[0].numpy(), currentModel.predict(x)[0]],
                1,
                "Single Step Prediction",
            )
        
        for x, y in dataset_val.take(1):
            predictionData = model.predict(x)
            denormalized_predictionData = denormalize(predictionData)[0]
            actualValue = y[0].numpy()
            print("predicted denormalized:", denormalized_predictionData)
            print("   actual denormalized:", denormalize(actualValue))
            print("             predicted:", predictionData[0])
            print("                actual:", actualValue)

# class myThread (threading.Thread):
#     def __init__(self, threadID, name, q):
#         threading.Thread.__init__(self)
#         self.threadID = threadID
#         self.name = name
#         self.q = q
#     def run(self):
#         print("Starting " + self.name)
#         train(self.name, self.q)
#         print("Exiting " + self.name)
            
# def train(threadName, q):
#     while not exitFlag:
#         queueLock.acquire()
#         if not workQueue.empty():
#             data = q.get()
#             queueLock.release()
#             print("%s processing %s" % (threadName, data))
#         else:
#             queueLock.release()
#         print(switcherOuts.get(q.get()))
#         with switcherOuts.get(q.get()):
#             switcherModels.get(q.get()).fit(
#                 dataset_train,
#                 epochs=epochs,
#                 validation_data=dataset_val,
#                 callbacks=[es_callback,modelckpt_callback, lr_scheduler]
#             )

# threadList = ["Thread-1", "Thread-2", "Thread-3"]

# # define number of models to be processed
# numberList = [];
# numberOfModels = 2

# # define maximum queue
# maxQueue = 10

# for i in range(numberOfModels):
#     numberList.append(i+1)
# queueLock = threading.Lock()
# workQueue = queue.Queue(maxQueue)
# threads = []
# threadID = 1

# # Create new threads
# for tName in threadList:
#     thread = myThread(threadID, tName, workQueue)
#     thread.start()
#     threads.append(thread)
#     threadID += 1

# # Fill the queue
# queueLock.acquire()
# for number in numberList:
#     workQueue.put(number)
# queueLock.release()

# # Wait for queue to empty
# while not workQueue.empty():
#     pass

# # Notify threads it's time to exit
# exitFlag = 1

# # Wait for all threads to complete
# for t in threads:
#     t.join()
# print("Exiting Main Thread")